# Grover's Algorithm

The **Grover's Search** quantum kata is a series of exercises designed
to get you familiar with Grover's search algorithm.

It covers the following topics:

* writing oracles for Grover's search,
* performing steps of the algorithm, and
* putting it all together: Grover's search algorithm.

*Reading material:*

* The tasks follow the explanation from *Quantum Computation and Quantum Information* by Nielsen and Chuang.
  In the 10th anniversary edition, this is section 6.1.2 on pages 248-251.
* A different explanation of Grover's algorithm can be found in 
  [this Wikipedia article](https://en.wikipedia.org/wiki/Grover%27s_algorithm).
* [An Introduction to Quantum Algorithms](https://people.cs.umass.edu/~strubell/doc/quantum_tutorial.pdf) by Emma Strubell, pages 20-24.
* [Lecture 4: Grover's Algorithm](https://www.cs.cmu.edu/~odonnell/quantum15/lecture04.pdf) by John Wright.
* Lectures [12](https://cs.uwaterloo.ca/~watrous/LectureNotes/CPSC519.Winter2006/12.pdf) and [13](https://cs.uwaterloo.ca/~watrous/LectureNotes/CPSC519.Winter2006/13.pdf) by John Watrous.
* [This page](http://davidbkemp.github.io/animated-qubits/grover.html) has an animated demonstration of Grover's algorithm for a simple case.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blanks (marked with the `// ...` comments)
with some Q# code that solves the task.  To verify your answer, run the cell with Ctrl/⌘+Enter.

Within each section, tasks are given in approximate order of increasing difficulty;
harder ones are marked with asterisks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [2]:
%package Microsoft.Quantum.Katas::0.9.1908.2906

Microsoft.Quantum.Standard::0.9.1908.2906, Microsoft.Quantum.Katas::0.9.1908.2906

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Oracles for Grover's Search


### Task 1.1. The $|11...1\rangle$ Oracle
**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of the target qubit (i.e., apply an X gate to it)
if the query register is in the $|11...1\rangle$ state,
and leave it unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

* If the query register is in state $|00...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|10...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|11...1\rangle$, flip the target qubit.

* If the query register is in state $\frac{1}{\sqrt{2}} \big(|00...0\rangle + |11...1\rangle \big)$, and the target is in state $|0\rangle$,
the joint state of the query register and the target qubit should be $\frac{1}{\sqrt{2}} \big(|00...00\rangle + |11...11\rangle \big)$.

In [3]:
%kata T11_Oracle_AllOnes_Test 

operation Oracle_AllOnes (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
    Controlled X(queryRegister, target);
}

Success!

### Task 1.2. The $|1010...\rangle$ Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

  Flip the state of the target qubit if the query register is in the $|1010...\rangle$ state;
that is, the state with alternating 1 and 0 values, with any number of qubits in the register.
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

 * If the register is in state $|0000000\rangle$, leave the target qubit unchanged.
 * If the register is in state $|10101\rangle$, flip the target qubit.

In [ ]:
%kata T12_Oracle_AlternatingBits_Test 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 1.3. Arbitrary Bit Pattern Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. A bit pattern of length N represented as `Bool[]`

**Goal:**

  Flip the state of the target qubit if the query register is in the state described by the given bit pattern
(`true` represents qubit state One, and `false` represents Zero).
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Example:**

  If the bit pattern is `[true, false]`, you need to flip the target qubit if and only if the qubits are in the $|10\rangle$ state.

In [34]:
%kata T13_Oracle_ArbitraryPattern_Test 

operation Oracle_ArbitraryPattern (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj {
    // ...
    let off = FindOffBits(queryRegister, pattern);
    
    within {
        ApplyToEachA(X, off);
    }
    apply {
        Controlled X(queryRegister, target);
    }
    
}

Success!

In [18]:
function FindOffBits(queryRegister : Qubit[], pattern : Bool[]) : Qubit[] {
    mutable ctrls = new Qubit[Length(queryRegister)];
    mutable count = 0;
    for (i in Microsoft.Quantum.Arrays.IndexRange(pattern)) {
        if (pattern[i] == false){
            set ctrls w/= count <- queryRegister[i];
            set count += 1;
        }
    }

    return ctrls[0..count-1];
}

FindOffBits

### Task 1.4. Oracle Converter

**Input:**

A marking oracle: an oracle that takes a register and a target qubit and
flips the target qubit if the register satisfies a certain condition.

**Output:**

A phase-flipping oracle: an oracle that takes a register and
flips the phase of the register if it satisfies this condition.

> Grover's algorithm relies on the search condition implemented as a phase-flipping oracle,
but it is often easier to write a marking oracle for a given condition. This transformation
allows to convert one type of oracle into the other. The transformation is described in the 
[Wikipedia article on Grover's algorithm](https://en.wikipedia.org/wiki/Grover%27s_algorithm), in the section "Description of ${U_\omega}$".

<br/>
<details>
  <summary>Need a hint? Click here </summary>
    Remember that you can define auxiliary operations. To do that, you'll need to create an extra code cell for each new operation and execute it before returning to this cell.
</details>

In [26]:

operation _impl (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit
is Adj {

    using (target = Qubit()) {
        // Put the target into the |-⟩ state
        X(target);
        H(target);

        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        markingOracle(register, target);

        // Put the target back into |0⟩ so we can return it
        H(target);
        X(target);
    }
}


_impl

In [27]:
%kata T14_OracleConverter_Test 

function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return _impl(markingOracle, _);
}

Success!

## Part II. The Grover Iteration

### Task 2.1. The Hadamard Transform

**Input:** A register of N qubits in an arbitrary state

**Goal:** Apply the Hadamard transform to each of the qubits in the register.

> If the register started in the $|0...0\rangle$ state, this operation
will prepare an equal superposition of all $2^{N}$ basis states.

In [ ]:
%kata T21_HadamardTransform_Test 

operation HadamardTransform (register : Qubit[]) : Unit is Adj {
    // ...
}

### Task 2.2. Conditional Phase Flip

**Input:**  A register of N qubits in an arbitrary state.

**Goal:**  Flip the sign of the state of the register if it is not in the $|0...0\rangle$ state.

**Examples:**

 * If the register is in state $|0...0\rangle$, leave it unchanged.

 * If the register is in any other basis state, multiply its phase by -1.

> This operation implements operator $2|0...0\rangle\langle0...0| - I$.

<br/>
<details>
  <summary>Need a hint? Click here </summary>
    Note that quantum states are defined up to a global phase.
    Thus the state obtained as a result of this operation is the same
    as the state obtained by flipping the sign of only the $|0...0\rangle$ state.
</details>
<br/>
<details>
  <summary>Need another hint? Click here </summary>
    You can use the same trick as in the oracle converter task.
</details>

In [30]:
%kata T22_ConditionalPhaseFlip_Test 


operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    // ...
    let oracle = OracleConverter(Oracle_AllZeros);
    oracle(register);
    
}

Success!

In [29]:

operation Oracle_AllZeros (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
    within {
        ApplyToEachA(X, queryRegister);
    }
    apply{
        Controlled X(queryRegister, target);
    }
}

Oracle_AllZeros

### Task 2.3. The Grover Iteration

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A phase-flipping oracle that takes an N-qubit register and flips
     the phase of the state if the register is in the desired state.

**Goal:**  Perform one Grover iteration.

<br/>
<details>
  <summary>Need a hint? Click here </summary>
    A Grover iteration consists of 4 steps:
    <ol>
    <li>Apply the Oracle</li>
    <li>Apply the Hadamard transform</li>
    <li>Perform a conditional phase shift</li>
    <li>Apply the Hadamard transform again</li>
    </ol>
</details>

In [32]:
%kata T23_GroverIteration_Test 

operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    // ...
    oracle(register);
    
    within{
        ApplyToEachA(H, register);
    }
    apply {
        ConditionalPhaseFlip(register);
    }
}

Success!

## Part III. Putting It All Together: Grover's Search Algorithm

### Task 3.1. Grover's Search

**Inputs:**

  1. N qubits in the $|0...0\rangle$ state.

  2. A marking oracle.

  3. The number of Grover iterations to perform.

**Goal:** Use Grover's algorithm to leave the register in the state that is marked by the oracle as the answer (with high probability).

> The number of iterations is passed as a parameter because it is defined by the nature of the problem
and is easier to configure/calculate outside the search algorithm itself (for example, in the classical driver).

In [33]:
%kata T31_GroversSearch_Test 

operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    // ...
    ApplyToEachA(H, register);
    
    let phaseOracle = OracleConverter(oracle);
    
    for (i in 1 .. iterations) {
        GroverIteration(register, phaseOracle);
    }
    
}

Success!

### Task 3.2. Using Grover's Search

**Goal:**   Use your implementation of Grover's Algorithm from Task 3.1 and the oracles from part 1
  to find the marked elements of the search space. This task is not covered by a test and allows you to experiment with running the algorithm.
  
> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details closed>
  <summary>Hint #1</summary>
    To check whether the algorithm found the correct answer (i.e., an answer marked as 1 by the oracle),
    you can apply the oracle once more to the register after you've measured it and an ancilla qubit,
    which will calculate the function of the answer found by the algorithm.
</details>
<br/>
<details closed>
  <summary>Hint #2</summary>
    Experiment with the number of iterations to see how it affects
    the probability of the algorithm finding the correct answer.
</details>
<br/>
<details closed>
  <summary>Hint #3</summary>
    You can use the Message function to output the results.
</details>

In [ ]:
operation Run_GroversSearch_Algorithm () : Unit {
    // ...
}

In [ ]:
%simulate Run_GroversSearch_Algorithm